In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "classify-to-learn"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.v2 import ExtendedEncoder, Encoder

In [4]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split

trainU = ImageDataset(dataset=datasets.TRAINING_UNLABELED,network_input_size=224, cuda=cuda)
unlabeled = DataLoader(trainU,128,True)

In [5]:

device = torch.device('cpu')
#PATH = "Storage\models\AutoEncoderModelV2.pth"
PATH = 'Storage\models\\finetunedResNet50.pth'

if cuda:
    model = torch.load(PATH)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
else:    
    model = torch.load(PATH, map_location=torch.device('cpu') )


In [106]:
all_preds = []
all_out = []
model.eval()
with torch.no_grad():
    for inputs,labels in tqdm(unlabeled):
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        print('outputs: ', outputs)
        print('outputs: ', outputs.shape)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_out.extend(outputs.cpu().numpy())
        
all_preds = np.array(all_preds)
all_out = np.array(all_out)


  0%|          | 1/887 [00:11<2:49:41, 11.49s/it]

outputs:  tensor([[-5.0114, -5.0258, -3.8712,  ..., -4.7319, -3.0730, -2.4333],
        [-5.5485, -3.5288, -3.7015,  ..., -2.0397, -3.7106, -5.6213],
        [-3.6674, -3.2510, -2.1828,  ..., -3.8474,  0.3089, -2.8785],
        ...,
        [-1.5525, -3.6200, -2.6248,  ..., -3.0427, -2.9700, -3.6160],
        [-3.6960, -3.6377, -1.2731,  ..., -3.6350, -3.0335, -3.0936],
        [-2.4984, -2.6898, -1.1263,  ..., -3.9730, -1.9684, -3.8488]])
outputs:  torch.Size([128, 251])


  0%|          | 1/887 [00:12<3:04:40, 12.51s/it]


KeyboardInterrupt: 

In [45]:
all_preds.tofile('all_preds.dat')

In [46]:
all_out.tofile('all_out.dat')

In [8]:
all_outfromfile = np.fromfile('all_out.dat')
print('all_outfromfile.shape: ', all_outfromfile.shape)
print('all_outfromfile: ', all_outfromfile[251*10:251*12])

all_predsfromfile = np.fromfile('all_preds.dat')
print('all_predsfromfile: ', all_predsfromfile)

all_outfromfile.shape:  (14238602,)
all_outfromfile:  [-2.16569939e+01 -1.34433041e+01 -5.17071166e+02 -1.43445950e+01
 -1.38958613e-03 -3.18893957e+00 -1.14185593e+02 -3.86089504e-01
 -1.23228833e-01 -4.21506117e+01 -6.66457397e+02 -4.27430649e+01
 -1.75377922e+01 -1.20305821e-01 -1.83984955e+02 -4.62129090e+02
 -2.63294637e-01 -7.62124390e+02 -2.66907307e-07 -1.67226677e+01
 -5.81665268e+01 -8.95967364e-01 -1.81731992e-02 -5.78819439e-02
 -1.21963242e+02 -2.86823235e+01 -4.62751329e-01 -4.01715468e-03
 -1.09553661e+01 -7.63213486e-02 -1.08947739e-01 -4.27984848e+01
 -1.46961269e+01 -1.37301139e-03 -7.95226289e+01 -6.83934758e-08
 -1.23356857e+02 -5.46397934e+01 -8.97421837e+00 -3.25734176e+01
 -2.52721672e+01  7.24990822e-09 -4.58425507e-02 -1.39261566e+02
 -1.18041592e+01 -1.16705403e-01 -9.53617454e-01 -1.37329922e+01
 -1.09394919e-02 -1.75242475e-03 -2.93954125e+01 -7.38595367e-01
 -5.97236918e+00 -1.25414562e+01 -1.31029644e+01 -3.24816345e+02
 -4.28959885e+01 -1.43961811e+01 -8.

In [13]:
for i in range(0, len(all_outfromfile), 251):
    out_proba_of_one_pic=all_outfromfile[i:i+251]
    most_proba = np.argmax(out_proba_of_one_pic)
    prob = all_outfromfile[i+most_proba]
    print('most_proba: ', most_proba)
    print('prob: ', prob/np.mean(out_proba_of_one_pic))

most_proba:  87
prob:  -2.6497350486825488e-06
most_proba:  159
prob:  -0.5659459085581671
most_proba:  186
prob:  -0.010728278136611539
most_proba:  198
prob:  -0.027931972326261024
most_proba:  220
prob:  -0.5962241004091421
most_proba:  169
prob:  -64.43859632929417
most_proba:  182
prob:  -4.512462281482816
most_proba:  11
prob:  -0.012138386176061902
most_proba:  31
prob:  -0.27820476664747507
most_proba:  135
prob:  -3.325374045549305e-05
most_proba:  141
prob:  -0.016618275977928816
most_proba:  43
prob:  -0.05997343861925295
most_proba:  13
prob:  -0.4863162546667628
most_proba:  133
prob:  -1.4329834334982137
most_proba:  146
prob:  -2.1632042137876257
most_proba:  13
prob:  -0.3780760309669975
most_proba:  214
prob:  -0.012287885755247538
most_proba:  237
prob:  -9.771932913245114e-09
most_proba:  43
prob:  -0.00041922055721551645
most_proba:  132
prob:  -0.1077401481996653
most_proba:  187
prob:  -23.158552046098546
most_proba:  107
prob:  -5.43124005683947
most_proba:  61
p

In [107]:
print("a", np.argmax(all_outfromfile[0:251]), "b", torch.argmax(outputs[0]))

a 87 b tensor(35)
